# 12 - DuckDB: SQL Analytics + Time Series + LLM Analysis

## 🎯 Objetivos
- Introducción a DuckDB (SQL analytics in-process)
- Análisis de series temporales
- Forecasting con Prophet y ARIMA
- Integración con LLMs para análisis de datos
- Visualizaciones avanzadas

## 📚 Tecnologías
- **DuckDB**: SQL OLAP database engine
- **Prophet**: Time series forecasting (Facebook)
- **statsmodels**: ARIMA, statistical models
- **OpenAI/Anthropic**: LLM para análisis
- **Plotly**: Visualizaciones interactivas

## ⭐ Complejidad: Básico/Intermedio

## 1. Instalación y Setup

In [ ]:
# Instalar dependencias
!pip install duckdb pandas numpy matplotlib seaborn plotly prophet statsmodels scikit-learn -q

In [ ]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print(f"✅ DuckDB version: {duckdb.__version__}")
print(f"✅ Pandas version: {pd.__version__}")

## 2. Introducción a DuckDB

DuckDB es un sistema de base de datos OLAP SQL in-process, similar a SQLite pero optimizado para analytics.

In [ ]:
# Crear conexión a DuckDB (en memoria)
con = duckdb.connect(':memory:')

# También podríamos crear una base de datos persistente:
# con = duckdb.connect('mi_base_datos.duckdb')

print("✅ Conexión a DuckDB establecida")

# Verificar versión
version = con.execute("SELECT version()").fetchone()
print(f"📊 DuckDB version: {version[0]}")

## 3. Operaciones Básicas con DuckDB

In [ ]:
# Crear tabla desde SQL
con.execute("""
    CREATE TABLE empleados (
        id INTEGER,
        nombre VARCHAR,
        edad INTEGER,
        departamento VARCHAR,
        salario DECIMAL(10,2)
    )
""")

# Insertar datos
con.execute("""
    INSERT INTO empleados VALUES
        (1, 'Juan', 28, 'TI', 50000),
        (2, 'María', 35, 'Ventas', 75000),
        (3, 'Pedro', 42, 'TI', 60000),
        (4, 'Ana', 29, 'Marketing', 55000),
        (5, 'Luis', 38, 'Ventas', 70000)
""")

# Query simple
result = con.execute("SELECT * FROM empleados").df()
print("📊 Tabla empleados:")
print(result)

# Agregación
result = con.execute("""
    SELECT 
        departamento,
        COUNT(*) as empleados,
        AVG(edad) as edad_promedio,
        AVG(salario) as salario_promedio
    FROM empleados
    GROUP BY departamento
    ORDER BY salario_promedio DESC
""").df()

print("\n📊 Resumen por departamento:")
print(result)

## 4. Integración DuckDB + Pandas

DuckDB puede consultar DataFrames de Pandas directamente.

In [ ]:
# Crear DataFrame de Pandas
productos_df = pd.DataFrame({
    'producto_id': [1, 2, 3, 4, 5],
    'nombre': ['Laptop', 'Mouse', 'Teclado', 'Monitor', 'Audífonos'],
    'precio': [1200, 25, 75, 350, 80],
    'categoria': ['Electrónica', 'Accesorios', 'Accesorios', 'Electrónica', 'Accesorios']
})

print("📊 DataFrame de Pandas:")
print(productos_df)

# Query directo sobre DataFrame de Pandas
result = con.execute("""
    SELECT 
        categoria,
        COUNT(*) as productos,
        AVG(precio) as precio_promedio,
        SUM(precio) as precio_total
    FROM productos_df
    GROUP BY categoria
""").df()

print("\n📊 Query sobre DataFrame de Pandas:")
print(result)

# También podemos usar la sintaxis alternativa
result = duckdb.query("""
    SELECT * FROM productos_df WHERE precio > 50 ORDER BY precio DESC
""").df()

print("\n📊 Filtrado con DuckDB:")
print(result)

## 5. Generar Datos de Time Series

Creamos datos sintéticos de series temporales para análisis.

In [ ]:
# Generar serie temporal de ventas
np.random.seed(42)

# Fecha inicial: 2 años atrás
start_date = datetime.now() - timedelta(days=730)
dates = pd.date_range(start=start_date, periods=730, freq='D')

# Componentes de la serie temporal:
# 1. Tendencia creciente
trend = np.linspace(100, 300, len(dates))

# 2. Estacionalidad semanal
weekly_seasonality = 30 * np.sin(2 * np.pi * np.arange(len(dates)) / 7)

# 3. Estacionalidad anual
yearly_seasonality = 50 * np.sin(2 * np.pi * np.arange(len(dates)) / 365)

# 4. Ruido aleatorio
noise = np.random.normal(0, 15, len(dates))

# 5. Eventos especiales (picos)
special_events = np.zeros(len(dates))
# Black Friday, Cyber Monday, etc.
event_days = [60, 150, 240, 330, 425, 515, 605, 695]
for day in event_days:
    if day < len(dates):
        special_events[day:day+3] = 100

# Combinar todos los componentes
sales = trend + weekly_seasonality + yearly_seasonality + noise + special_events
sales = np.maximum(sales, 0)  # No ventas negativas

# Crear DataFrame
ts_df = pd.DataFrame({
    'fecha': dates,
    'ventas': sales,
    'tendencia': trend,
    'estacionalidad_semanal': weekly_seasonality,
    'estacionalidad_anual': yearly_seasonality,
    'ruido': noise
})

# Agregar features temporales
ts_df['año'] = ts_df['fecha'].dt.year
ts_df['mes'] = ts_df['fecha'].dt.month
ts_df['dia_semana'] = ts_df['fecha'].dt.dayofweek
ts_df['dia_mes'] = ts_df['fecha'].dt.day
ts_df['trimestre'] = ts_df['fecha'].dt.quarter

print(f"📊 Serie temporal creada: {len(ts_df)} días")
print(f"📊 Rango: {ts_df['fecha'].min()} a {ts_df['fecha'].max()}")
print(f"\n📊 Primeras filas:")
print(ts_df.head(10))
print(f"\n📊 Estadísticas:")
print(ts_df[['ventas', 'tendencia']].describe())

## 6. Análisis SQL de Time Series con DuckDB

In [ ]:
# Análisis mensual
monthly_analysis = con.execute("""
    SELECT 
        año,
        mes,
        COUNT(*) as dias,
        ROUND(AVG(ventas), 2) as ventas_promedio,
        ROUND(SUM(ventas), 2) as ventas_total,
        ROUND(MIN(ventas), 2) as ventas_min,
        ROUND(MAX(ventas), 2) as ventas_max,
        ROUND(STDDEV(ventas), 2) as ventas_stddev
    FROM ts_df
    GROUP BY año, mes
    ORDER BY año, mes
""").df()

print("📊 Análisis Mensual:")
print(monthly_analysis.head(12))

# Análisis por día de la semana
weekly_pattern = con.execute("""
    SELECT 
        dia_semana,
        CASE dia_semana
            WHEN 0 THEN 'Lunes'
            WHEN 1 THEN 'Martes'
            WHEN 2 THEN 'Miércoles'
            WHEN 3 THEN 'Jueves'
            WHEN 4 THEN 'Viernes'
            WHEN 5 THEN 'Sábado'
            WHEN 6 THEN 'Domingo'
        END as dia_nombre,
        ROUND(AVG(ventas), 2) as ventas_promedio,
        COUNT(*) as registros
    FROM ts_df
    GROUP BY dia_semana
    ORDER BY dia_semana
""").df()

print("\n📊 Patrón Semanal:")
print(weekly_pattern)

# Crecimiento mensual
growth = con.execute("""
    WITH monthly AS (
        SELECT 
            año,
            mes,
            SUM(ventas) as ventas_total
        FROM ts_df
        GROUP BY año, mes
    )
    SELECT 
        año,
        mes,
        ventas_total,
        LAG(ventas_total) OVER (ORDER BY año, mes) as ventas_mes_anterior,
        ROUND(
            (ventas_total - LAG(ventas_total) OVER (ORDER BY año, mes)) / 
            LAG(ventas_total) OVER (ORDER BY año, mes) * 100, 
            2
        ) as crecimiento_pct
    FROM monthly
    ORDER BY año, mes
""").df()

print("\n📊 Crecimiento Mensual:")
print(growth.head(12))

## 7. Visualización de Time Series

In [ ]:
# Visualización con Plotly (interactivo)
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Ventas Diarias', 'Componentes de la Serie', 'Distribución'),
    specs=[[{"secondary_y": False}],
           [{"secondary_y": False}],
           [{"secondary_y": False}]],
    vertical_spacing=0.1
)

# Serie temporal completa
fig.add_trace(
    go.Scatter(x=ts_df['fecha'], y=ts_df['ventas'], 
               name='Ventas', line=dict(color='blue', width=1)),
    row=1, col=1
)

# Tendencia
fig.add_trace(
    go.Scatter(x=ts_df['fecha'], y=ts_df['tendencia'], 
               name='Tendencia', line=dict(color='red', width=2, dash='dash')),
    row=1, col=1
)

# Componentes
fig.add_trace(
    go.Scatter(x=ts_df['fecha'], y=ts_df['estacionalidad_semanal'], 
               name='Est. Semanal', line=dict(color='green')),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=ts_df['fecha'], y=ts_df['estacionalidad_anual'], 
               name='Est. Anual', line=dict(color='orange')),
    row=2, col=1
)

# Histograma
fig.add_trace(
    go.Histogram(x=ts_df['ventas'], name='Distribución', 
                 marker=dict(color='purple'), nbinsx=50),
    row=3, col=1
)

fig.update_layout(height=900, title_text="Análisis de Serie Temporal", showlegend=True)
fig.show()

print("✅ Gráficos interactivos generados")

In [ ]:
# Patrón semanal visualizado
fig = px.bar(weekly_pattern, x='dia_nombre', y='ventas_promedio',
             title='Ventas Promedio por Día de la Semana',
             labels={'dia_nombre': 'Día', 'ventas_promedio': 'Ventas Promedio'},
             color='ventas_promedio',
             color_continuous_scale='Blues')
fig.show()

# Crecimiento mensual
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(range(len(growth))),
    y=growth['crecimiento_pct'],
    mode='lines+markers',
    name='Crecimiento %',
    line=dict(color='green', width=2)
))
fig.add_hline(y=0, line_dash="dash", line_color="red")
fig.update_layout(
    title='Crecimiento Mensual (%)',
    xaxis_title='Mes',
    yaxis_title='Crecimiento %',
    height=400
)
fig.show()

## 8. Forecasting con Prophet

Prophet es una librería de Facebook para forecasting de series temporales.

In [ ]:
from prophet import Prophet

# Preparar datos para Prophet (requiere columnas 'ds' y 'y')
prophet_df = ts_df[['fecha', 'ventas']].copy()
prophet_df.columns = ['ds', 'y']

# Crear y entrenar modelo
print("🚀 Entrenando modelo Prophet...")
model_prophet = Prophet(
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=True,
    seasonality_mode='additive'
)

model_prophet.fit(prophet_df)
print("✅ Modelo Prophet entrenado")

# Crear dataframe para predicciones futuras (90 días)
future = model_prophet.make_future_dataframe(periods=90)
print(f"📊 Prediciendo {90} días futuros...")

# Hacer predicciones
forecast = model_prophet.predict(future)

print("\n📊 Predicciones:")
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(10))

In [ ]:
# Visualizar predicciones con Prophet
from prophet.plot import plot_plotly, plot_components_plotly

# Gráfico de predicción
fig1 = plot_plotly(model_prophet, forecast)
fig1.update_layout(title='Predicción con Prophet (90 días)', height=500)
fig1.show()

# Componentes de la serie
fig2 = plot_components_plotly(model_prophet, forecast)
fig2.update_layout(height=800)
fig2.show()

print("✅ Visualizaciones de Prophet generadas")

## 9. Forecasting con ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# Preparar serie temporal
ts_series = ts_df.set_index('fecha')['ventas']

# Descomposición de la serie
print("📊 Descomposición de la serie temporal...")
decomposition = seasonal_decompose(ts_series, model='additive', period=7)

# Visualizar componentes
fig, axes = plt.subplots(4, 1, figsize=(14, 10))

decomposition.observed.plot(ax=axes[0], title='Serie Original')
decomposition.trend.plot(ax=axes[1], title='Tendencia')
decomposition.seasonal.plot(ax=axes[2], title='Estacionalidad')
decomposition.resid.plot(ax=axes[3], title='Residuos')

plt.tight_layout()
plt.show()

print("✅ Descomposición completada")

In [ ]:
# Entrenar modelo ARIMA
print("🚀 Entrenando modelo ARIMA...")

# ARIMA(p,d,q) - usaremos (1,1,1) como ejemplo simple
model_arima = ARIMA(ts_series, order=(1, 1, 1))
fitted_model = model_arima.fit()

print("✅ Modelo ARIMA entrenado")
print("\n📊 Resumen del modelo:")
print(fitted_model.summary())

# Predicciones
forecast_steps = 90
forecast_arima = fitted_model.forecast(steps=forecast_steps)

print(f"\n📊 Primeras 10 predicciones ARIMA:")
print(forecast_arima.head(10))

In [ ]:
# Visualizar predicciones ARIMA
plt.figure(figsize=(14, 6))

# Últimos 180 días de datos reales
plt.plot(ts_series[-180:], label='Datos Reales', color='blue')

# Predicciones
forecast_index = pd.date_range(
    start=ts_series.index[-1] + timedelta(days=1),
    periods=forecast_steps,
    freq='D'
)
plt.plot(forecast_index, forecast_arima, label='Predicción ARIMA', 
         color='red', linestyle='--')

plt.title('Predicción con ARIMA (90 días)', fontsize=14)
plt.xlabel('Fecha')
plt.ylabel('Ventas')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("✅ Visualización ARIMA completada")

## 10. Comparación de Modelos

In [ ]:
# Comparar predicciones de Prophet vs ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Obtener predicciones de Prophet para los mismos días
forecast_future = forecast[forecast['ds'] > ts_df['fecha'].max()].head(90)

# Crear visualización comparativa
fig = go.Figure()

# Datos históricos (últimos 180 días)
fig.add_trace(go.Scatter(
    x=ts_df['fecha'][-180:],
    y=ts_df['ventas'][-180:],
    mode='lines',
    name='Datos Históricos',
    line=dict(color='blue', width=2)
))

# Predicción Prophet
fig.add_trace(go.Scatter(
    x=forecast_future['ds'],
    y=forecast_future['yhat'],
    mode='lines',
    name='Prophet',
    line=dict(color='green', width=2, dash='dash')
))

# Intervalo de confianza Prophet
fig.add_trace(go.Scatter(
    x=forecast_future['ds'],
    y=forecast_future['yhat_upper'],
    mode='lines',
    name='Prophet Upper',
    line=dict(color='green', width=0),
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=forecast_future['ds'],
    y=forecast_future['yhat_lower'],
    mode='lines',
    name='Prophet Lower',
    line=dict(color='green', width=0),
    fill='tonexty',
    fillcolor='rgba(0,255,0,0.1)',
    showlegend=False
))

# Predicción ARIMA
fig.add_trace(go.Scatter(
    x=forecast_index,
    y=forecast_arima,
    mode='lines',
    name='ARIMA',
    line=dict(color='red', width=2, dash='dot')
))

fig.update_layout(
    title='Comparación: Prophet vs ARIMA',
    xaxis_title='Fecha',
    yaxis_title='Ventas',
    height=600,
    hovermode='x unified'
)

fig.show()

print("✅ Comparación de modelos visualizada")

## 11. Almacenar Resultados en DuckDB

In [ ]:
# Crear DataFrame con predicciones
predictions_df = pd.DataFrame({
    'fecha': forecast_index,
    'prediccion_prophet': forecast_future['yhat'].values,
    'prophet_lower': forecast_future['yhat_lower'].values,
    'prophet_upper': forecast_future['yhat_upper'].values,
    'prediccion_arima': forecast_arima.values
})

# Registrar en DuckDB
con.execute("CREATE TABLE IF NOT EXISTS ts_data AS SELECT * FROM ts_df")
con.execute("CREATE TABLE IF NOT EXISTS predictions AS SELECT * FROM predictions_df")

print("✅ Datos guardados en DuckDB")

# Query combinada
result = con.execute("""
    SELECT 
        p.fecha,
        p.prediccion_prophet,
        p.prediccion_arima,
        ROUND((p.prediccion_prophet + p.prediccion_arima) / 2, 2) as prediccion_promedio
    FROM predictions p
    ORDER BY fecha
    LIMIT 10
""").df()

print("\n📊 Predicciones combinadas:")
print(result)

## 12. Análisis con LLM (Generación de Insights)

Usaremos un LLM para analizar automáticamente los datos y generar insights.

In [ ]:
# Preparar resumen de datos para el LLM
def generate_data_summary():
    summary = f"""
    RESUMEN DE DATOS DE VENTAS
    ===========================
    
    Período: {ts_df['fecha'].min()} a {ts_df['fecha'].max()}
    Total de días: {len(ts_df)}
    
    ESTADÍSTICAS GENERALES:
    - Ventas promedio diarias: ${ts_df['ventas'].mean():.2f}
    - Ventas mínimas: ${ts_df['ventas'].min():.2f}
    - Ventas máximas: ${ts_df['ventas'].max():.2f}
    - Desviación estándar: ${ts_df['ventas'].std():.2f}
    
    VENTAS POR DÍA DE LA SEMANA:
    {weekly_pattern.to_string(index=False)}
    
    TENDENCIA:
    - Ventas primer mes: ${monthly_analysis.iloc[0]['ventas_total']:.2f}
    - Ventas último mes: ${monthly_analysis.iloc[-1]['ventas_total']:.2f}
    - Crecimiento total: {((monthly_analysis.iloc[-1]['ventas_total'] / monthly_analysis.iloc[0]['ventas_total']) - 1) * 100:.2f}%
    
    PREDICCIONES (próximos 30 días):
    - Prophet: ${forecast_future['yhat'].head(30).mean():.2f} promedio
    - ARIMA: ${forecast_arima.head(30).mean():.2f} promedio
    """
    return summary

data_summary = generate_data_summary()
print(data_summary)

In [ ]:
# Función para análisis con LLM (simulado)
def analyze_with_llm(data_summary):
    """
    En producción, aquí conectarías con OpenAI, Anthropic, u otro LLM.
    Por ahora, generamos un análisis de ejemplo.
    
    Para usar un LLM real, descomenta y configura:
    
    # import openai
    # openai.api_key = 'tu-api-key'
    # response = openai.ChatCompletion.create(
    #     model="gpt-4",
    #     messages=[
    #         {"role": "system", "content": "Eres un analista de datos experto."},
    #         {"role": "user", "content": f"Analiza estos datos: {data_summary}"}
    #     ]
    # )
    # return response.choices[0].message.content
    """
    
    # Análisis simulado basado en los datos
    insights = f"""
    🤖 ANÁLISIS GENERADO POR LLM
    ============================
    
    📊 INSIGHTS CLAVE:
    
    1. TENDENCIA POSITIVA
       Las ventas muestran una tendencia creciente sostenida a lo largo del período analizado.
       El crecimiento total sugiere un negocio en expansión.
    
    2. ESTACIONALIDAD SEMANAL
       Se observa un patrón claro en los días de la semana. Los días con mayores ventas
       son {weekly_pattern.nlargest(1, 'ventas_promedio')['dia_nombre'].values[0]}, 
       mientras que {weekly_pattern.nsmallest(1, 'ventas_promedio')['dia_nombre'].values[0]} 
       tiene las ventas más bajas.
    
    3. VARIABILIDAD
       La desviación estándar indica una variabilidad moderada en las ventas diarias,
       lo cual es normal en negocios retail.
    
    4. PREDICCIONES
       Ambos modelos (Prophet y ARIMA) predicen una continuación de la tendencia positiva.
       Se recomienda:
       - Aumentar inventario para los próximos 30 días
       - Preparar campañas de marketing para días de menor venta
       - Monitorear eventos especiales que generan picos de venta
    
    5. RECOMENDACIONES
       ✅ Optimizar staffing para días de alta demanda
       ✅ Implementar promociones en días de baja venta
       ✅ Analizar factores externos que causan los picos de venta
       ✅ Considerar expansión dada la tendencia positiva
    
    📈 FORECAST SUMMARY:
       Los próximos 90 días muestran expectativas de crecimiento continuo.
       El modelo Prophet sugiere mayor confianza con intervalos más estrechos.
    """
    
    return insights

# Generar análisis
llm_analysis = analyze_with_llm(data_summary)
print(llm_analysis)

## 13. Guardar Análisis LLM en DuckDB

In [ ]:
# Crear tabla de análisis
analysis_record = pd.DataFrame([{
    'fecha_analisis': datetime.now(),
    'periodo_inicio': ts_df['fecha'].min(),
    'periodo_fin': ts_df['fecha'].max(),
    'ventas_promedio': ts_df['ventas'].mean(),
    'prediccion_30d_prophet': forecast_future['yhat'].head(30).mean(),
    'prediccion_30d_arima': forecast_arima.head(30).mean(),
    'insights_llm': llm_analysis
}])

con.execute("CREATE TABLE IF NOT EXISTS llm_analysis AS SELECT * FROM analysis_record")

print("✅ Análisis LLM guardado en DuckDB")

# Recuperar análisis
stored_analysis = con.execute("""
    SELECT 
        fecha_analisis,
        ROUND(ventas_promedio, 2) as ventas_promedio,
        ROUND(prediccion_30d_prophet, 2) as pred_prophet,
        ROUND(prediccion_30d_arima, 2) as pred_arima
    FROM llm_analysis
""").df()

print("\n📊 Análisis almacenado:")
print(stored_analysis)

## 14. Dashboard Resumen con DuckDB

In [ ]:
# Query compleja para dashboard
dashboard_query = """
WITH stats AS (
    SELECT 
        COUNT(*) as total_dias,
        ROUND(AVG(ventas), 2) as ventas_avg,
        ROUND(SUM(ventas), 2) as ventas_total,
        ROUND(MIN(ventas), 2) as ventas_min,
        ROUND(MAX(ventas), 2) as ventas_max
    FROM ts_data
),
weekly_avg AS (
    SELECT ROUND(AVG(ventas), 2) as ventas_semana_actual
    FROM ts_data
    WHERE fecha >= (SELECT MAX(fecha) - INTERVAL '7 days' FROM ts_data)
),
forecast_avg AS (
    SELECT 
        ROUND(AVG(prediccion_prophet), 2) as forecast_prophet,
        ROUND(AVG(prediccion_arima), 2) as forecast_arima
    FROM predictions
    WHERE fecha <= (SELECT MIN(fecha) + INTERVAL '30 days' FROM predictions)
)
SELECT 
    s.total_dias,
    s.ventas_avg,
    s.ventas_total,
    s.ventas_min,
    s.ventas_max,
    w.ventas_semana_actual,
    f.forecast_prophet as prediccion_30d_prophet,
    f.forecast_arima as prediccion_30d_arima,
    ROUND((w.ventas_semana_actual - s.ventas_avg) / s.ventas_avg * 100, 2) as variacion_vs_promedio_pct
FROM stats s, weekly_avg w, forecast_avg f
"""

dashboard_data = con.execute(dashboard_query).df()

print("📊 DASHBOARD EJECUTIVO")
print("=" * 60)
print(f"\n📈 MÉTRICAS GENERALES:")
print(f"   Días analizados: {dashboard_data['total_dias'][0]}")
print(f"   Ventas totales: ${dashboard_data['ventas_total'][0]:,.2f}")
print(f"   Ventas promedio: ${dashboard_data['ventas_avg'][0]:,.2f}")
print(f"   Rango: ${dashboard_data['ventas_min'][0]:.2f} - ${dashboard_data['ventas_max'][0]:.2f}")

print(f"\n📊 SEMANA ACTUAL:")
print(f"   Promedio últimos 7 días: ${dashboard_data['ventas_semana_actual'][0]:,.2f}")
print(f"   Variación vs promedio histórico: {dashboard_data['variacion_vs_promedio_pct'][0]:+.2f}%")

print(f"\n🔮 PREDICCIONES (próximos 30 días):")
print(f"   Prophet: ${dashboard_data['prediccion_30d_prophet'][0]:,.2f} promedio diario")
print(f"   ARIMA: ${dashboard_data['prediccion_30d_arima'][0]:,.2f} promedio diario")

print("\n" + "=" * 60)

## 15. Exportar Resultados

In [ ]:
# Exportar a Parquet (formato eficiente)
con.execute("COPY ts_data TO 'timeseries_data.parquet' (FORMAT PARQUET)")
con.execute("COPY predictions TO 'predictions.parquet' (FORMAT PARQUET)")

print("✅ Datos exportados a Parquet")

# Exportar a CSV
predictions_df.to_csv('predictions.csv', index=False)
print("✅ Predicciones exportadas a CSV")

# Guardar base de datos DuckDB en archivo
con.execute("EXPORT DATABASE 'timeseries_backup' (FORMAT PARQUET)")
print("✅ Base de datos completa exportada")

## 16. Resumen y Mejores Prácticas

### ✅ Conceptos Clave:
1. **DuckDB**: SQL analytics in-process, ideal para data science
2. **Time Series**: Análisis temporal con tendencias y estacionalidad
3. **Prophet**: Forecasting robusto con componentes interpretables
4. **ARIMA**: Modelo estadístico clásico para series temporales
5. **LLM Analysis**: Generación automática de insights

### 💡 Ventajas de DuckDB:
- ✅ Extremadamente rápido para queries analíticos
- ✅ Integración perfecta con Pandas/NumPy
- ✅ No requiere servidor (in-process)
- ✅ Soporte completo SQL con ventanas, CTEs, etc.
- ✅ Formato columnar eficiente (Parquet nativo)
- ✅ Ideal para notebooks y análisis exploratorio

### 💡 Time Series Mejores Prácticas:
- ✅ Identifica componentes: tendencia, estacionalidad, ruido
- ✅ Usa múltiples modelos y compara resultados
- ✅ Valida con datos out-of-sample
- ✅ Considera eventos especiales y outliers
- ✅ Actualiza modelos regularmente con nuevos datos

### 🚀 Próximos Pasos:
- Deep Learning para time series (LSTM, Transformers)
- Anomaly detection en tiempo real
- Multi-variate forecasting
- AutoML para time series (AutoARIMA, NeuralProphet)
- Integración con Airflow para pipelines automatizados

In [ ]:
# Cerrar conexión DuckDB
con.close()

print("🎉 Tutorial de DuckDB + Time Series + LLM completado!")
print("\n📊 Resumen:")
print("   - Serie temporal de 730 días analizada")
print("   - 2 modelos de forecasting entrenados (Prophet, ARIMA)")
print("   - 90 días de predicciones generadas")
print("   - Insights generados con LLM")
print("   - Datos almacenados en DuckDB")
print("\n📁 Archivos generados:")
print("   - timeseries_data.parquet")
print("   - predictions.parquet")
print("   - predictions.csv")
print("   - timeseries_backup/ (DuckDB export)")
print("\n✅ Conexión DuckDB cerrada")